<a href="https://colab.research.google.com/github/JosephPLim/FishListExclusives/blob/main/Fish_List_Exclusives.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# <center> Fish List Exclusives</center>
## <center> Joseph Lim</center>
### <center> v0.1 </center>
This version of Fish List Exclusives takes two spreadsheets of data, and compares the fish by scientific name, to see which fish are exclusive to each list.

For v 0.1: Two spreadsheets were compared, and merged into a third spreadsheet, which shows the # of fish and their price side by side.

**How does this improve on the previous version?**

This improves the readability of the project for the end user. It also further helps the user productively figure out if a fish is exclusive to a shipping list part of the time. Spreadsheets are now cleaned slightly in order to better facilitate programming.

**What still needs to be improved?**

This still does not account for typos or wrong scientific names.
* typos
* outdated and mismatched scientific names and
* sizing of fish (S/M/L, etc)

**Next step (v0.2)**

Incorporate fish sizing, age where possible.

In [1]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
bali_4_filename = "/content/drive/MyDrive/Colab Notebooks/Bali-4 stocklist.xlsx"
bali_4_df = pd.read_excel(bali_4_filename)
bali_4_df.head(0)

,Code,ORDER,Quantity,SCIENTIFIC NAME,COMMON NAME,SIZE,Price,SPECIAL,Unnamed: 8,Unnamed: 9


In [3]:
isptc_filename = "/content/drive/MyDrive/Colab Notebooks/ISPTC-Stocklist.xlsx"
isptc_df = pd.read_excel(isptc_filename)
isptc_df.head(0)

,CODE,QTY,ORDER,COMMON NAME,SCIENTIFIC NAME,PRICE


In [4]:
#Step 1: Trim the scientific names to only be two words
def trim_words(name):
  name = name.split()
  newname = " ".join(name[0:2])
  return newname.lower()

In [5]:
for i in range(len(isptc_df["SCIENTIFIC NAME"])):
  name = isptc_df.at[i, "SCIENTIFIC NAME"]
  name = trim_words(name)
  isptc_df.at[i, "SCIENTIFIC NAME"] = name

for i in range(len(bali_4_df["SCIENTIFIC NAME"])):
  name = bali_4_df.at[i,"SCIENTIFIC NAME"]
  name = trim_words(name)
  bali_4_df.at[i, "SCIENTIFIC NAME"] = name

Step 2: Clean the two spreadsheets. By this, the # in stock should be an int, and the price should be a float rounded to two decimal places. If the stock is NaN, it should be replaced with 0; same with price. If a dictionary for spell check is in place, then use it here.

In [6]:
#cleaning ISPTC
isptc_df.dtypes

CODE                object
QTY                float64
ORDER               object
COMMON NAME         object
SCIENTIFIC NAME     object
PRICE              float64
dtype: object

In [7]:
isptc_df = isptc_df.drop(columns = ["CODE", "ORDER"])
isptc_df.head()

,QTY,COMMON NAME,SCIENTIFIC NAME,PRICE
0,150.0,BROWN FLAT ANEMONES,radianthus species,1.2
1,100.0,WHITE FLAT ANEMONES,radianthus species,1.8
2,70.0,BROWN CARPET (M),stoichactis kenti,3.5
3,20.0,BROWN CARPET (L),stoichactis kenti,3.6
4,8.0,GREEN COLOR CARPET ANEMONE,stoichactis gigantium,15.0


In [8]:
isptc_df.isna().sum()

QTY                0
COMMON NAME        0
SCIENTIFIC NAME    0
PRICE              0
dtype: int64

In [9]:
#cleaning Bali-4
bali_4_df.dtypes

Code                object
ORDER               object
Quantity            object
SCIENTIFIC NAME     object
COMMON NAME         object
SIZE                object
Price              float64
SPECIAL             object
Unnamed: 8         float64
Unnamed: 9          object
dtype: object

In [10]:
bali_4_df = bali_4_df.drop(columns = ["Code", "ORDER", "Unnamed: 8", "Unnamed: 9"])

In [11]:
bali_4_df.isna().sum()

Quantity             0
SCIENTIFIC NAME      0
COMMON NAME          0
SIZE               225
Price                1
SPECIAL            299
dtype: int64

In [12]:
#for special: ignore. Later on: impute with the price of the cell
bali_4_df["SIZE"].fillna("NA", inplace = True)
bali_4_df.dropna(subset = ["Price"], inplace = True)

In [13]:
#for row in df: if special is not a float, special = price
for ind in bali_4_df.index:
  special = bali_4_df["SPECIAL"][ind]
  price = bali_4_df["Price"][ind]
  if special == " ":
    bali_4_df["SPECIAL"][ind] = bali_4_df["Price"][ind]
  elif special <= 0:
    bali_4_df["SPECIAL"][ind] = bali_4_df["Price"][ind]
#  elif pd.isna(bali_4_df["SPECIAL"][ind]):
#    bali_4_df["SPECIAL"][ind] = bali_4_df["Price"][ind]
  #if pd.isna(bali_4_df["SPECIAL"][ind]):
  #  print(bali_4_df["SPECIAL"][ind])
bali_4_df.dtypes


<ipython-input-13-8f0dc7445626>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bali_4_df["SPECIAL"][ind] = bali_4_df["Price"][ind]


Quantity            object
SCIENTIFIC NAME     object
COMMON NAME         object
SIZE                object
Price              float64
SPECIAL             object
dtype: object

In [14]:
bali_4_df["SPECIAL"] = bali_4_df["SPECIAL"].astype(float)

Now the scientific names *sorta* match.

We want to read all of spreadsheet A into a different DataFrame with the following data dictionary/layout

| Common Name | Scientific Name | # in Stock A | price A | # in stock B | price B |
|-------------|-----------------|--------------|---------|--------------|---------|


*  common name = name in English
*  scientific name = name in Latin (standardized)
* \# in stock A = # of fish in stock from spreadsheet A
* price A = price of a single fish of that species from spreadsheet A
* \# in stock B = # of fish in stock from spreadsheet B
* price B = price of a single fish of that species from spreadsheet B




After that, we want to put all of spreadsheet B into the same spreadsheet, but if the scientific name already exists, compare to the commmon name, and if they match: add secondary columns for [x,y,z] values

If a fish is not in both spreadsheets, the corresponding spreadsheet values should be 0.00 for price and 0 for stock

Finally, the spreadsheet should be alphabetized by scientific name, and export it

In [15]:
#read all of spreadsheet A into a different dataframe
exportdf = pd.DataFrame()
isptc_df.sort_values("SCIENTIFIC NAME", inplace = True)
isptc_df.head()

,QTY,COMMON NAME,SCIENTIFIC NAME,PRICE
103,200.0,ORANGE LINE CHROMIS,acanthochromis polyacanthus,0.2
273,12.0,MIMIC HALF BLACK TANG,acanthurus chronixis,1.6
260,100.0,POWDER BROWN TANG,acanthurus japonicus,2.8
254,10.0,LINEATUS TANG (L),acanthurus lineatus,6.0
253,40.0,LINEATUS TANG (M),acanthurus lineatus,3.0


In [16]:
exportdf["Common Name"]= isptc_df["COMMON NAME"]
exportdf["Scientific Name"] = isptc_df["SCIENTIFIC NAME"]
exportdf["ISPTC #"] = isptc_df["QTY"]
exportdf["ISPTC Price"] = isptc_df["PRICE"]
exportdf["Bali 4 #"] = None
exportdf["Bali 4 Price"] = 0
exportdf.sort_values("Scientific Name", inplace = True)
exportdf.head()

,Common Name,Scientific Name,ISPTC #,ISPTC Price,Bali 4 #,Bali 4 Price
103,ORANGE LINE CHROMIS,acanthochromis polyacanthus,200.0,0.2,None,0
273,MIMIC HALF BLACK TANG,acanthurus chronixis,12.0,1.6,None,0
260,POWDER BROWN TANG,acanthurus japonicus,100.0,2.8,None,0
254,LINEATUS TANG (L),acanthurus lineatus,10.0,6.0,None,0
253,LINEATUS TANG (M),acanthurus lineatus,40.0,3.0,None,0


In [17]:
#After that, we want to put all of spreadsheet B into the same spreadsheet, 
#but if the scientific name already exists, compare to the commmon name, 
#and if they match: add secondary columns for [x,y,z] values
bali_4_df.head()

,Quantity,SCIENTIFIC NAME,COMMON NAME,SIZE,Price,SPECIAL
0,30.0,apolemichthys trimaculatus,FLAGFIN ANGEL (M/L),M/L,9.9,7.92
1,4.0,centropyge bicolor,BICOLOR ANGEL,S,1.5,1.20
2,113.0,centropyge bicolor,BICOLOR ANGEL,M/L,1.5,1.20
3,3.0,centropyge bispinosa,CORAL BEAUTY ANGEL,S,8.0,NaN
4,2.0,centropyge bispinosa,CORAL BEAUTY ANGEL,M/L,8.0,NaN


In [18]:
newDict = exportdf.to_dict()
scientificNames = newDict["Scientific Name"].values()
scientificNames = set(scientificNames) #allows us to run this in O(1)

In [19]:
#goes through bali 4, makes a list to append to the bottom of the row
#initializes list of rows that need to be added into the exportdf bali 4 and price areas
currentlyExists = []
for ind in bali_4_df.index:
  row = []
  row.append(bali_4_df["COMMON NAME"][ind])
  row.append(bali_4_df["SCIENTIFIC NAME"][ind])

  if row[1] not in scientificNames:
    row.append(0)
    row.append(0)

  row.append(bali_4_df["Quantity"][ind])
  if bali_4_df["SPECIAL"][ind] > 0: #account for NaNs
    row.append(round(bali_4_df["SPECIAL"][ind],2))
  else:
    row.append(round(bali_4_df["Price"][ind],2))
  
  if row[1] in scientificNames:
    currentlyExists.append(row)
  else:
    exportdf.loc[len(exportdf.index)] = row
exportdf = exportdf.sort_values("Scientific Name", ignore_index = True)
#the ignore_index = True allows the index to be relabelled
#allows searchsorted to actually work.

In [20]:
exportdf.head()

,Common Name,Scientific Name,ISPTC #,ISPTC Price,Bali 4 #,Bali 4 Price
0,SERGEANT MAYOR,abudefduf sexfasciatus,0.0,0.0,154.0,0.16
1,ORANGE LINE CHROMIS,acanthochromis polyacanthus,200.0,0.2,NaN,0.00
2,YELLOW BACKFIN SURGEONSFISH,acanthurus bariene,0.0,0.0,21.0,2.52
3,YELLOW BACKFIN SURGEONSFISH,acanthurus bariene,0.0,0.0,6.0,4.00
4,MIMIC HALF BLACK TANG,acanthurus chronixis,12.0,1.6,NaN,0.00


In [21]:
exportdf["Scientific Name"].head()

0         abudefduf sexfasciatus
1    acanthochromis polyacanthus
2             acanthurus bariene
3             acanthurus bariene
4           acanthurus chronixis
Name: Scientific Name, dtype: object

In [22]:
#create searchsort numpy array
#turn the column into a numpy array
scinameSeries = exportdf["Scientific Name"].squeeze()
scinameSeries.searchsorted("centropyge bicolor")

88

In [23]:
exportdf["Scientific Name"][scinameSeries.searchsorted("centropyge bicolor")]

'centropyge bicolor'

In [24]:
currentlyExists[1]

['BICOLOR ANGEL', 'centropyge bicolor', 113.0, 1.2]

In [25]:
#conduct binary search for the scientific name
prev = ""
for row in currentlyExists:
  sciname = row[1]
  if sciname != prev:
    ind = scinameSeries.searchsorted(sciname)
    exportdf.at[ind, "Bali 4 #"] = row[2]
    exportdf.at[ind, "Bali 4 Price" ]= row[3]
    prev = sciname
  #if there are duplicates, take the first price for now. Fix later on.
exportdf.head()

,Common Name,Scientific Name,ISPTC #,ISPTC Price,Bali 4 #,Bali 4 Price
0,SERGEANT MAYOR,abudefduf sexfasciatus,0.0,0.0,154.0,0.16
1,ORANGE LINE CHROMIS,acanthochromis polyacanthus,200.0,0.2,NaN,0.00
2,YELLOW BACKFIN SURGEONSFISH,acanthurus bariene,0.0,0.0,21.0,2.52
3,YELLOW BACKFIN SURGEONSFISH,acanthurus bariene,0.0,0.0,6.0,4.00
4,MIMIC HALF BLACK TANG,acanthurus chronixis,12.0,1.6,NaN,0.00


In [27]:
#Alphabetize the spreadsheet by scientific name:
#https://stackoverflow.com/questions/43401903/python-order-dataframe-alphabetically
exportdf = exportdf.sort_values("Scientific Name", ignore_index = True)

exportdf.to_excel(excel_writer='mergedOrderLists.xlsx',index = False)